In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
df_a = pd.read_csv('./data/amz_com-ecommerce_sample.csv', encoding= 'unicode_escape')

In [ ]:
df_f = pd.read_csv('./data/flipkart_com-ecommerce_sample.csv', encoding= 'unicode_escape')

In [ ]:
df_a.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
df_f.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [ ]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [ ]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

# Text pre-processing

In [ ]:
df_a.product_name = df_a.product_name.str.lower()

In [ ]:
df_a.shape

(20000, 15)

In [ ]:
df_f.product_name = df_f.product_name.str.lower()

In [ ]:
df_f.shape

(20000, 15)

In [ ]:
# analysing product name randomly from the datset
idx = np.random.randint(0, df_a.shape[0])
df_a.loc[idx, 'product_name']

"layer'r shot compact shock,impact and turbo body spray (pack of 3) combo set"

In [ ]:
# analysing product name randomly from the datset
idx = np.random.randint(0, df_f.shape[0])
df_f.loc[idx, 'product_name']

'fashblush forever new glam alloy necklace'

In [ ]:
# dropping duplicates records from the data
df_a.drop_duplicates(subset='product_name', inplace=True)

In [ ]:
df_a.shape

(12628, 15)

In [ ]:
# dropping duplicates records from the data
df_f.drop_duplicates(subset='product_name', inplace=True)

In [ ]:
df_f.shape

(12623, 15)

In [ ]:
# resetting index
df_a.reset_index(drop=True, inplace=True)

In [ ]:
df_f.reset_index(drop=True, inplace=True)

In [ ]:
# function to preprocess text
def preprocess(text):
    stemmer = PorterStemmer()
    text = ' '.join(stemmer.stem(word) for word in text.split())
    text = re.sub(r'[^a-zA-Z]', ' ', text).lower()  # consider only alphabets
    text = ' '.join(text.split()) # to remove spaces between the text
    return text

In [ ]:
df_f.image.fillna('https://logicacloud.eu/wp-content/themes/logica/images/No-Image-Found-400x264.png', inplace=True)
df_a.image.fillna('https://logicacloud.eu/wp-content/themes/logica/images/No-Image-Found-400x264.png', inplace=True)

In [ ]:
# function to clean links
def clean_image_links(link):
    link = link.strip('][').split(', ')[0]
    link = link.replace('"', '')
    return link

In [ ]:
# preprocessing the product name
df_a['product_name_processed'] = df_a.product_name.apply(preprocess)

In [ ]:
df_f['product_name_processed'] = df_f.product_name.apply(preprocess)

# tf-idf and cosine similarity
reference: <a href='https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630'> To understand the working of code </a> 

In [ ]:
df_a.shape

(12628, 16)

In [ ]:
df_f.shape

(12623, 16)

In [ ]:
# creating instace of TfidfVectorizer
tfidfvectoriser_both = TfidfVectorizer()

In [ ]:
# merging both the datasets
df_both = pd.concat([df_a, df_f], axis=0)

In [ ]:
df_both.shape

(25410, 16)

In [ ]:
# fitting the vectorizer on the entire corpus
tfidfvectoriser_both.fit(df_both.product_name_processed)

TfidfVectorizer()

In [ ]:
# creating vector representation of amazon dataset
tfidf_a = tfidfvectoriser_both.transform(df_a.product_name_processed)

In [ ]:
tfidf_a.shape # (no. of records, embedding for each sentence)

(12734, 9140)

In [ ]:
# creating vector representation of amazon dataset
tfidf_f = tfidfvectoriser_both.transform(df_f.product_name_processed)

In [ ]:
tfidf_f.shape

(12676, 9140)

In [ ]:
# calculating pairwise similarity for each flipkart product with respect to every amazon product
ps_fa = np.dot(tfidf_f, tfidf_a.T).toarray()

In [ ]:
# calculating pairwise similarity for each amazon product with respect to every flipkart product
ps_af = np.dot(tfidf_a, tfidf_f.T).toarray()

In [ ]:
ps_fa.shape

(12676, 12734)

In [ ]:
ps_af.shape

(12734, 12676)

In [ ]:
# pick random product from amazon dataset
df_a.loc[823, 'product_name']

'Shortkut enterprises Model no 434 Mobile/Tablet Speaker'

In [ ]:
# provide index of your selected product
np.argsort(ps_af[342])[::-1] # always exclude very first index (similarity with respect to itself)

array([ 341,  342, 6588, ..., 7713, 7712, 6337])

In [ ]:
# considering second element from above result to find similar product in the flipkart dataset
df_f.loc[341, 'product_name']

"Tenn Casual, Party Sleeveless Floral Print Women's Top"

# Finding similar product based on input

#### find in amazon dataset

In [ ]:
# take product name from flipkart dataset
inp = df_f.loc[66, 'product_name'] # try replacing 899 with other index
print(inp)

TEN TEN Women's Black Knee Length Boots Boots


In [ ]:
# preprocess the input
inp_f = preprocess(inp)
print(inp_f)

ten ten women black knee length boot boot


In [ ]:
# convert the text into vector embedding
embed = tfidfvectoriser_both.transform([inp_f])

In [ ]:
# 1 sentence has vector embedding of size 9141
embed.shape 

(1, 9140)

In [ ]:
# find the cosine similarity of our input sent with all sentence embedd of amazon
res = np.dot(embed, tfidf_a.T) 

**Note:** `For finding similar product in flipkart : np.dot(embed, tfidf_f.T)`

In [ ]:
# select the index which has highest similarity
idx = np.argmax(res) 

In [ ]:
# display the most similar product from amazon dataset
df_a.loc[idx, 'product_name']

"ten ten women's black knee length boots boots"

In [ ]:
# saving dataframes into csv
df_a.to_csv('./data/amazon_process_data.csv', index=False)
df_f.to_csv('./data/flipkart_process_data.csv', index=False)

# Saving the tokenzier and data embeddings

In [ ]:
file_a = './tfidf_vectors/amazon_data_embeddings.pkl'
file_f = './tfidf_vectors/flipkart_data_embeddings.pkl'
file_vec = './tfidf_vectors/input_vector.pkl'

In [ ]:
from joblib import load, dump

In [ ]:
# saving amazon_data_embeddings
with open(file_a, 'wb') as f:
    dump(tfidf_a, f)

In [ ]:
# saving amazon_data_embeddings
with open(file_f, 'wb') as f:
    dump(tfidf_f, f)

In [ ]:
# saving amazon_data_embeddings
with open(file_vec, 'wb') as f:
    dump(tfidfvectoriser_both, f)